In [19]:
# https://huggingface.co/docs/transformers/main/en/tasks/masked_language_modeling

from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DatasetMIDI, DataCollator
from transformers import DataCollatorForLanguageModeling
from miditok.utils import split_files_for_training
from torch.utils.data import DataLoader
from pathlib import Path
from random import shuffle

from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer

In [20]:
# Load the tokenizer
prefix = '/media/maindisk/maximos/data/GiantMIDI-PIano/midis_v1.2/aug/midis'
saved_tokenizer_path = f'{prefix}_REMI_BPE_tokenizer.json'
path_to_dataset = prefix
path_to_train_splits = f'{prefix}_splits_REMI_BPE/train/midis'
path_to_valid_splits = f'{prefix}_splits_REMI_BPE/valid/midis'

max_seq_len = 1024

tokenizer = REMI(params=Path(saved_tokenizer_path))

tokenizer.pad_token = tokenizer.special_tokens[0]
tokenizer.mask_token = tokenizer.special_tokens[1]

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15
)

In [21]:
tokenizer.pad_token = tokenizer.special_tokens[0]


In [22]:
files_paths = list(Path(path_to_dataset).glob("**/*.mid"))
shuffle(files_paths)

total_num_files = len(files_paths)
num_files_valid = round(total_num_files * 0.10)

midi_paths_valid = files_paths[:num_files_valid]
midi_paths_train = files_paths[num_files_valid:]

In [23]:
train_dataset = DatasetMIDI(
    files_paths=Path(path_to_train_splits).glob('**/*.mid'),
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
valid_dataset = DatasetMIDI(
    files_paths=Path(path_to_valid_splits).glob('**/*.mid'),
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
# collator = DataCollator(pad_token_id=tokenizer.pad_token_id, copy_inputs_as_labels=True)
# dataloader = DataLoader(dataset, batch_size=64, collate_fn=collator)


In [24]:
print(train_dataset[0])

{'input_ids': tensor([   2, 1312,  185,  ..., 2052, 8328,  217])}


In [25]:
model = AutoModelForMaskedLM.from_pretrained("distilbert/distilroberta-base")

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
training_args = TrainingArguments(
    output_dir='models/robertaGM',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

AttributeError: 'REMI' object has no attribute 'convert_tokens_to_ids'

In [26]:
from transformers import AutoTokenizer

In [27]:
tokenizer_lang = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [32]:
tokenizer.get_special_tokens_mask = tokenizer_lang.get_special_tokens_mask

In [28]:
tokenizer.pad = tokenizer_lang.pad